# 导入包

In [29]:
from collections import defaultdict
from tqdm import tqdm
import re
import json
from copy import deepcopy
import random

import torch
import os
import pickle
from transformers import AutoTokenizer
import transformers
import torch
import torch.nn as nn
import math
from transformers import AutoModel
tokenizer=AutoTokenizer.from_pretrained("/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/",do_lower_case=False)
import time
import pandas as pd
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample
from collections import Counter
from copy import deepcopy
from nlp_basictasks.modules.transformers import BertTokenizer,BertModel,BertConfig
from nlp_basictasks.modules import MLP
device=torch.device('cuda')

# 读取大规模的知识图谱

In [2]:
sub_map = defaultdict(list)
obj_map = defaultdict(list)
so_map=defaultdict(list)
sp_map=defaultdict(list)
kg_path='../knowledge/Knowledge.txt'
with open(kg_path) as f:
    lines=f.readlines()
    
entities_set=set()
relations_set=set()
print(len(lines))

#new_triples=[]
for i in tqdm(range(len(lines))):
    line=lines[i]
    l = line.strip().split('\t')
    s = l[0].strip()
    p = l[1].strip()
    o = l[2].strip()
    if s=='' or p=='' or o=='':
        print('-'*100)
        continue
    entities_set.add(s)
    entities_set.add(o)
    relations_set.add(p)
    #new_triples.append((s,p,o))
    sub_map[s].append((p, o))
    obj_map[o].append((s, p))
    so_map[(s,o)].append(p)
    sp_map[(s,p)].append(o)

22883549


100%|███████████████████████████████████████████████████████████████████████████████████████| 22883549/22883549 [01:30<00:00, 253892.69it/s]


In [3]:
print('大规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(lines),len(entities_set),len(relations_set)))

大规模图谱中一共有22883549个三元组，9418104个实体，245837个关系


# 读取数据

In [4]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/train.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/dev.json'))
data.extend(read_data('/home/xhsun/Desktop/KG/kgCLUE/qa_data/test_public.json'))

原始数据有18000个样本
原始数据有2000个样本
原始数据有2000个样本


In [5]:
print("原始数据有{}个样本".format(len(data)))
for i in range(10):
    print(data[i])

原始数据有22000个样本
{'id': 0, 'question': '你知道守望星光是哪家公司出品的吗？', 'answer': '守望星光（韩玉玲、刘东考演唱歌曲） ||| 出品公司 ||| 韩玉玲原创音乐'}
{'id': 1, 'question': '武汉交通职业学院计算机协会有哪些学校？', 'answer': '武汉交通职业学院计算机协会 ||| 学校 ||| 武汉交通职业学院'}
{'id': 2, 'question': '你知道龙晶用繁体字怎么写吗？', 'answer': '龙晶（游戏） ||| 繁体 ||| 龍晶'}
{'id': 3, 'question': '无罪无我的和声是什么？', 'answer': '无罪无我 ||| 和声 ||| 章韵、徐晨辰'}
{'id': 4, 'question': '你知道影舞者的车身有多重吗？', 'answer': '影舞者（《QQ飞车》中的赛车） ||| 车身重量 ||| 1232kg'}
{'id': 5, 'question': '冯慧专业是什么方向？', 'answer': '冯慧（武汉大学教授） ||| 专业方向 ||| 偏微分方程数值解'}
{'id': 6, 'question': '你知道戴姆勒股份公司有多少员工吗？', 'answer': '戴姆勒股份公司 ||| 员工数 ||| 298655 人(2020年)'}
{'id': 7, 'question': '谁知道魔力w.i.t.c.h.系列有哪些丛书？', 'answer': '魔力w.i.t.c.h. ||| 丛书系列 ||| 魔力w.i.t.c.h.100少女完全手册、魔力w.i.t.c.h.故事集、魔力w.i.t.c.h.小说《破碎的魔球》'}
{'id': 8, 'question': '谁知道端午节有什么重要意义？', 'answer': '端午节（中国四大传统节日之一） ||| 节日意义 ||| 传承与弘扬非物质文化'}
{'id': 9, 'question': '谁唱的风不息？', 'answer': '风不息（1995年郭富城发行的音乐专辑） ||| 歌手 ||| 郭富城'}


# 将数据转成question-relation的形式

In [21]:
all_examples=[]
new_relations=set()
for example in data:
    question,answer=example['question'],example['answer']
    triplet=answer.strip().split('|||')
    assert len(triplet)==3
    assert type(question)==str
    question=question.strip()
    entity=triplet[0].strip()
    rel=triplet[1].strip()
    ans=triplet[2].strip()
    assert rel in relations_set#确保关系在KG中
    try:
        assert ans in entities_set and entity in entities_set#确保实体在KG中
    except:
        print(example)
    new_relations.add(rel)
    for r in so_map[(entity,ans)]:
        new_relations.add(r)
    
    tmp=set()
    for p,o in sub_map[entity]:
        new_relations.add(p)#添加与头实体有相连路径的所有关系
        tmp.add(p)
    all_examples.append({"question":question,"head":entity,"relation":rel,"tail":ans,'other_relations':list(tmp)})

{'id': 5591, 'question': '姚明的中文名是什么？', 'answer': '王子（亚洲篮球联合会主席、中国篮球协会主席） ||| 中文名 ||| 王子'}
{'id': 13703, 'question': '能告诉我西藏大学具体是什么类型的大学吗？,<triple id=14704>', 'answer': '西藏大学 ||| 属性 ||| 211工程 中西部高校基础能力建设工程 西藏自治区人民政府与教育部共建高校'}


In [23]:
len(new_relations)

14013

In [24]:
all_examples[0]

{'question': '你知道守望星光是哪家公司出品的吗？',
 'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
 'relation': '出品公司',
 'tail': '韩玉玲原创音乐',
 'other_relations': ['发行日期',
  '音乐风格',
  '填词',
  '谱曲',
  '出品公司',
  '中文名',
  '歌曲时长',
  '歌曲原唱',
  '歌曲语言',
  '导演',
  '所属专辑']}

**现在已经有了rel2id，每一个样本也都有对应的关系和relation_range**

In [57]:
rel2id={}
for i,key in enumerate(new_relations):
    rel2id[key]=len(rel2id)

In [58]:
rel2id

{'无线电呼号': 0,
 '武器类别': 1,
 '额高': 2,
 '总库容': 3,
 '政治党派': 4,
 '布阵人': 5,
 '村委会驻地': 6,
 '横幅': 7,
 '党籍': 8,
 '旋转速度': 9,
 '发射性能': 10,
 '幼儿': 11,
 '创办人': 12,
 '乐队身份': 13,
 '票房': 14,
 '网络端口': 15,
 '招生批次': 16,
 '实现': 17,
 '标志建筑': 18,
 '学科分支': 19,
 '姓氏名人': 20,
 '数据格式': 21,
 '车辆亮点': 22,
 '著名学生': 23,
 '比例': 24,
 '外消旋体熔点': 25,
 '制作成本': 26,
 '兴盛时期': 27,
 '满载排水量：': 28,
 '嫂子': 29,
 '参赛队伍': 30,
 '图书建设': 31,
 '起止日期': 32,
 '艺术总监': 33,
 '调味品': 34,
 '起飞重量': 35,
 '拉丁全称': 36,
 '封装尺寸': 37,
 '下级': 38,
 '等等': 39,
 '主要用于': 40,
 '社训': 41,
 '代表作品': 42,
 '出场时间': 43,
 '满分': 44,
 '游戏平台': 45,
 '房间数量': 46,
 '球队颜色': 47,
 '攻击': 48,
 '服役期间': 49,
 '所属大学': 50,
 '相关国内组织': 51,
 '地区:': 52,
 '自身硬件': 53,
 '最佳旅游时间': 54,
 '活跃年份': 55,
 '百科名片': 56,
 '行李舱容积': 57,
 '实务书籍': 58,
 '元件': 59,
 '开始': 60,
 '政府组织': 61,
 '单层规格': 62,
 '品类': 63,
 '红岛': 64,
 '作品作者': 65,
 '具备能力': 66,
 '出发地': 67,
 '其他变体': 68,
 '选择条件': 69,
 '欣赏的歌手': 70,
 '使用介质': 71,
 '意思': 72,
 '首任沙皇': 73,
 '人均耕地': 74,
 '主屏像素密度': 75,
 '现演播室': 76,
 '办公所在地': 77,
 '舰载机：': 78,
 '展馆地点': 7

# 模型定义

In [70]:
class RelationExtract(nn.Module):
    def __init__(self, model_path,
                num_labels,
                state_dict=None,
                is_finetune=False,
                pooling_type='cls'):
        super().__init__()
        self.num_labels=num_labels
        self.pooling_type=pooling_type
        if is_finetune==False:
            #logger.info("Loading model from {}, which is from huggingface model".format(model_path))
            self.load_huggingface_model(bert_model_path=model_path)
        else:
            self.load_finetuned_model(model_path=model_path)
            #logger.info("Loading model from {}, which has been finetuned.".format(model_path))
            
    def load_huggingface_model(self,bert_model_path):
        self.config=BertConfig.from_pretrained(bert_model_path)
        self.bert=BertModel.from_pretrained(bert_model_path)
        self.head_layer=MLP(in_features=self.config.hidden_size,out_features=self.num_labels)

    def load_finetuned_model(self,model_path):
        bert_save_path=os.path.join(model_path,"BERT")#save的时候将BERT保存在model_path下的BERT文件夹中
        self.config=BertConfig.from_pretrained(bert_save_path)
        self.bert=BertModel.from_pretrained(bert_save_path)

        head_save_path=os.path.join(model_path,'MLP')
        self.head_layer=MLP.load(input_path=head_save_path)

    def save(self,output_path):
        bert_save_path=os.path.join(output_path,"BERT")
        self.bert.save_pretrained(bert_save_path,save_config=False)#下面已经save，不用save两次，虽然没什么影响
        self.config.save_pretrained(bert_save_path)
        head_save_path=os.path.join(output_path,'MLP')
        self.head_layer.save(head_save_path)
        
    def forward(self,input_ids,attention_mask=None,token_type_ids=None,label_ids=None,relation_range=None,output_all_encoded_layers=False):
        '''
        input_ids.size()==attention_mask.size()==token_type_ids.size()==position_ids.size()==
        (batch_size,seq_length)
        label_ids.size()==(batch_size,self.num_labels)
        relation_range.size()==(batch_size,self.num_labels)，是一个one_hot向量，与问题中topic实体相连的关系的位置是1
        '''
        (sequence_outputs,pooled_output)=self.bert(input_ids=input_ids,
                                                  token_type_ids=token_type_ids,
                                                   attention_mask=attention_mask,
                                                  output_all_encoded_layers=False)
        #要注意到sequence_output[0]与pooled_output的区别在于pooled_output是经过一层tanh的
        assert len(pooled_output.size())==2 and pooled_output.size(1)==self.config.hidden_size

        if self.pooling_type=='cls':
            before_logits=pooled_output
        elif self.pooling_type=='last_layer':
            #取最后一层的mean pooling
            if output_all_encoded_layers==True:
                assert type(sequence_outputs)==list
                sequence_outputs=sequence_outputs[-1]
            #sequence_outputs现在代表last layer的hidden_states
            attention_mask_expanded=attention_mask.unsqueeze(-1).expand(sequence_outputs.size())
            #sequence_outputs.size()==(bsz,pad_seq_len,dim)==attention_mask_expanded.size()

            sum_mask=attention_mask.sum(1).unsqueeze(1)#(batch_size,1)
            sum_mask=torch.clamp(sum_mask,min=1e-7)
            before_logits=torch.sum(sequence_outputs*attention_mask_expanded,1)/sum_mask#(bsz,seq_len,dim)-->(bsz,dim)
        elif self.pooling_type=='last_two_layer':
            #print(output_all_encoded_layers,type(sequence_outputs))
            assert output_all_encoded_layers==True and type(sequence_outputs)==list
            attention_mask_expanded=attention_mask.unsqueeze(-1).expand(sequence_outputs[-1].size())
            #sequence_outputs.size()==(bsz,pad_seq_len,dim)==attention_mask_expanded.size()
            sum_mask=attention_mask.sum(1).unsqueeze(1)#(batch_size,1)
            sum_mask=torch.clamp(sum_mask,min=1e-7)
            #print(sequence_outputs[-1].size(),attention_mask_expanded.size())
            last_1_pooling=torch.sum(sequence_outputs[-1]*attention_mask_expanded,1)/sum_mask
            last_2_pooling=torch.sum(sequence_outputs[-2]*attention_mask_expanded,1)/sum_mask

            before_logits=(last_1_pooling+last_2_pooling)/2
        
        else:
            raise Exception("Unknown pooling type %s error"%self.pooling_type)
        
        logits=self.head_layer(before_logits)#(batch_size,num_labels)
        if not self.training:
            scores,idx=torch.max(logits,dim=1)#[batch_size],[batch_size]
            return {"predict_scores":scores,"predict_idx":idx}
        else:
            assert label_ids.size()==logits.size()
            if relation_range is not None:
                weight=label_ids*99+1#答案位置的值是100，其余所有位置的值是1
                weight=weight*relation_range#答案位置的值是100，与topic实体相连的所有关系的值是1，其余的关系值是0
            else:
                weight=torch.ones_like(label_ids)
                
            loss=torch.sum(weight*torch.pow(logits-label_ids,2))/torch.sum(weight)
            return {"loss":loss}

# 构造Dataloader

In [59]:
def collate(batch):
    batch = list(zip(*batch))
    question_features,relation_id_tensor,relation_range_tensor = batch
    relation_id_tensor=torch.stack(relation_id_tensor)#(batch_size,num_relations)
    question_features = {k:torch.cat([q[k] for q in question_features], dim=0) for k in question_features[0]}
    relation_range_tensor = torch.stack(relation_range_tensor)
    return question_features,relation_id_tensor,relation_range_tensor

class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, rel2id):
        self.data = data
        self.rel2id = rel2id

    def __getitem__(self, index):
        question_features,relation_id,relation_range = self.data[index]
        relation_id_tensor = self.toOneHot(relation_id)#[num_relations]
        relation_range_tensor = self.toOneHot(relation_range)#[num_relations]
        return question_features,relation_id_tensor,relation_range_tensor

    def __len__(self):
        return len(self.data)

    def toOneHot(self, indices):
        indices = torch.LongTensor(indices)
        vec_len = len(self.rel2id)
        one_hot = torch.FloatTensor(vec_len)
        one_hot.zero_()
        one_hot.scatter_(0, indices, 1)
        return one_hot
    
class DataLoaderForRE(torch.utils.data.DataLoader):
    def __init__(self,all_examples,label2id,tokenizer,batch_size,shuffle=False,max_length=32):
        self.tokenizer=tokenizer
        self.label2id=label2id
        
        data=[]
        for example in all_examples:
            question,relation,other_relations=example['question'],example['relation'],example['other_relations']
            relation_id=[label2id[relation]]
            relation_range=[label2id[rel] for rel in other_relations]
            question_features=self.tokenizer(question.strip(),max_length=max_length,padding="max_length",return_tensors='pt')
            data.append([question_features,relation_id,relation_range])
        print('data number: {}'.format(len(data)))
        
        dataset=Dataset(data=data,rel2id=label2id)
        super().__init__(
            dataset, 
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=collate, 
            )

In [60]:
train_dataloader=DataLoaderForRE(all_examples=all_examples,label2id=rel2id,tokenizer=tokenizer,batch_size=3,shuffle=False)

data number: 22000


In [71]:
len(rel2id)

14013

In [73]:
with open("../code/TransferNet/rel2id.json",'w') as f:
    json.dump(obj=rel2id,fp=f,ensure_ascii=False)

In [74]:
len(all_examples)

22000

In [76]:
random.shuffle(all_examples)
with open("../code/NERPlusRE/train_data.json","w") as f:
    for example in all_examples[:-2000]:
        f.write(json.dumps(example,ensure_ascii=False)+'\n')
with open("../code/NERPlusRE/test_data.json","w") as f:
    for example in all_examples[-2000:]:
        f.write(json.dumps(example,ensure_ascii=False)+'\n')

In [82]:
def toOntHot(indices):
    indices = torch.LongTensor(indices)
    vec_len = 15
    one_hot = torch.FloatTensor(vec_len)
    one_hot.zero_()
    one_hot.scatter_(0, indices, 1)
    return one_hot

label_ids=torch.stack([toOntHot([3]),toOntHot([5])])

In [83]:
label_ids

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [84]:
label_ids=torch.argmax(label_ids,dim=1)

In [85]:
label_ids

tensor([3, 5])

In [86]:
predict=torch.LongTensor([3,4])
predict

tensor([3, 4])

In [89]:
(predict==label_ids).float().mean()

tensor(0.5000)